In [1]:
# import libraries
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import datetime as dt

pd.options.display.max_rows = 15

# 1. Pandas的基本数据结构
pandas最重要的数据结构是Series和DataFrame  
Series可以理解为一维数组，和arrays或者lists的结构相似  
DataFrame可以理解为二维数组，由一系列的Series(列)构成。每一个Series内的数据，理论为同一个DataType，而DataFrame为这些Series有一个共用的index来搜寻位置。在时间序列数据里，index一般为时间。

## 1.1 Series
Series包含一个与数组关联的数据标签，被叫做索引。如果没有给数据指定索引，会设定 0 到 N-1 的默认索引。
### 建立
- 通过list直接建立(可以附加index)
- 通过dict建立，key默认为index。指定的index override keys。

In [3]:
# 通过list直接建立
# 可以附加index
s1 = Series([4, 7, -5, 3])
s2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

In [4]:
# 通过dict建立, key默认为index
# 如果指定了index, 则只保留overlap的部分，dict的key中没有的赋予NaN
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
s3 = Series(sdata)
states = ['California', 'Ohio', 'Texas', 'Oregon']
s4 = Series(sdata, states)
s4

California        NaN
Ohio          35000.0
Texas         71000.0
Oregon        16000.0
dtype: float64

### 关于索引
- `series.index`
- `series.values`，返回ndarray type
- 对象本身和它的索引都有一个`name`属性
- Series的索引可以通过赋值直接更改，但不能单个更改，只能全部更新；DataFrame亦然。

In [6]:
# 对象本身和它的索引都有一个name属性
s4.name = 'population'
s4.index.name = 'state'
s4

state
California        NaN
Ohio          35000.0
Texas         71000.0
Oregon        16000.0
Name: population, dtype: float64

### 定位
- 通过位置 `series[0]`
- 通过名称 `series['a']`，`series.a`

## 1.2 DataFrame
可以理解为一组共享同一个index的Series
### 建立
- 通过dict建立，key对应columns
- 双层dict，外层key对应columns，内层key对应index
- dict的values必须统一长度
- index的键入必须和dict的长度一样

In [14]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
       'year': [2000, 2001, 2002, 2001, 2002],
       'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}
df1 = DataFrame(data)

In [15]:
# 双层字典，外层key对应columns，内层key对应index
pop = {'Nevada': {2001:2.4, 2002:2.9},
      'Ohio': {2000:1.5, 2001:1.7, 2002:3.6}}
df2 = DataFrame(pop)
df2

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


### 关于索引


### 定位
- 列
    - `df['col_name']`
    - `df.col_name`
    - `df.ix[:, 'col_name']`
    - `df.ix[:, col_num]`
- 行
    - `df.ix['index_name']`
    - `df.ix[index_num]`
    - `ix`形式可以理解为忽略了对列的引用
- m行n列
    - `df.ix`形式
    - 层级形式`df['col_name']['row_name']`
- 取出的都是reference，不能对其中的元素进行直接修改  
> `b = df['year']`  
> `b[0] = 1`这样的形式不可以

### 赋值
- 列的赋值`df['col_name'] = obj`  
如果是数列的话必须长度一致。
- 行的赋值`df.ix[index] = obj`  
obj是Series形式，index对应到df中的columns

### 删除
- 列  
    - `del df['col_name']`彻底删除
    - `df.drop(cols,1)`只是创建一个新的备份
- 行
    - `df.drop(index_num/index_name`

## 1.3 索引
索引是不可变的

### 类型
类型|pandas中的主要索引对象
---|---
Index|最通用的索引对象，使用Python对象的NumPy数组来表示坐标轴标签
Int64Index|对整型值的特化索引
MultiIndex|"分层"索引对象，表示单个轴的多层次的索引
DatetimeIndex|存储纳秒时间戳(使用NumPy的datetime64 dtype来表示)
PeriodIndex|对周期数据(时间间隔的)的特化索引

每个索引都有许多关于集合逻辑的方法和属性，且能够解决它所包含的数据的常见问题。  

### method and attribute
function|索引的方法和属性
---|---
append|链接额外的索引对象，产生一个新的索引
diff|计算索引的差集
intersection|交集
union|并集
isin|得出一个布尔数表示每一个值是否在所传递的集合里
delete|计算删除位置i的元素的索引
drop|计算删除所传递的值后的索引
insert|计算在位置i插入元素后的索引
is_monotonie|返回True，如果每一个元素都比它前面的元素大或相等
is_unique|返回True，如果索引没有重复的值

### 重新索引  
`reindex`需要注意的是，`reindex`并不对原df进行操作，而是生成了一个新的
- 对值进行内插或填充 `ffill` or `bfill` or `fill_value`
- 可以一次对行与列同时进行重新索引，但差值只在行进行
- 使用带标签索引的`ix`可以把重新索引做的更简单

In [3]:
s1 = Series([4.5, 7.2, -5.3, 3.6], index=['d','b','a','c'])
s2 = s1.reindex(['a','b','c','c','e'])
s2

a   -5.3
b    7.2
c    3.6
c    3.6
e    NaN
dtype: float64

In [4]:
s3 = Series(['blue','purple','yellow'], index=[0,2,4])
s3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [5]:
# 列的重新索引
df = DataFrame(np.arange(9).reshape((3,3)), index=['a','c','d'], columns=['Ohio','Texas','California'])
states = ['Texas', 'Utah', 'California']
df.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [6]:
# 可以一次对行与列同时进行重新索引，但差值只在行进行
df.reindex(index=['a','b','c','d'], method='ffill', columns=states)

,Texas,Utah,California
a,1,NaN,2
b,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [7]:
# 使用带标签锁芯的'ix'可以把重新索引做的更简单
df.ix[['a','b','c','d'], states]

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


In [8]:
# missing

## 1.4 Time Series Index
(to be done)

In [3]:
dates = pd.date_range('20130101', periods=6)
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [4]:
prng = pd.period_range('1991Q1', '2004Q4', freq='Q-NOV')
ts = pd.Series(np.random.randn(len(prng)), prng)
ts.index[-1]
# type: pandas.peropd.Period
# attribute: year, month, day, no date

Period('2004Q4', 'Q-NOV')

In [5]:
ts.index = (prng.asfreq('M','e')+1).asfreq('H','s')+9
ts

1991-03-01 09:00    0.803729
1991-06-01 09:00   -0.549952
1991-09-01 09:00   -2.245404
1991-12-01 09:00    0.869041
1992-03-01 09:00   -0.508702
1992-06-01 09:00   -0.111956
1992-09-01 09:00   -1.075721
                      ...   
2003-06-01 09:00   -2.039267
2003-09-01 09:00   -0.747174
2003-12-01 09:00   -0.699560
2004-03-01 09:00   -1.472039
2004-06-01 09:00    0.436009
2004-09-01 09:00    1.800086
2004-12-01 09:00   -0.035231
Freq: H, dtype: float64

# 2. 一些数组操作


In [2]:
dates = pd.date_range('20130101',periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

In [3]:
# Transpose
df.T

,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00
A,0.684346,-0.442428,-0.406470,-1.011514,1.154496,-0.568598
B,-1.045700,-0.118350,2.594857,1.318265,-1.158101,0.528244
C,1.981283,0.856943,-1.069507,-0.640984,1.951000,-0.391605
D,-1.161247,0.003684,1.671639,-0.745967,-0.461955,0.638851


In [4]:
# Sort by an axis
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2013-01-01,-1.161247,1.981283,-1.045700,0.684346
2013-01-02,0.003684,0.856943,-0.118350,-0.442428
2013-01-03,1.671639,-1.069507,2.594857,-0.406470
2013-01-04,-0.745967,-0.640984,1.318265,-1.011514
2013-01-05,-0.461955,1.951000,-1.158101,1.154496
2013-01-06,0.638851,-0.391605,0.528244,-0.568598


In [5]:
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2013-01-06,-0.568598,0.528244,-0.391605,0.638851
2013-01-05,1.154496,-1.158101,1.951000,-0.461955
2013-01-04,-1.011514,1.318265,-0.640984,-0.745967
2013-01-03,-0.406470,2.594857,-1.069507,1.671639
2013-01-02,-0.442428,-0.118350,0.856943,0.003684
2013-01-01,0.684346,-1.045700,1.981283,-1.161247


In [7]:
# Sort by values
df.sort_values(by='B')

,A,B,C,D
2013-01-05,1.154496,-1.158101,1.951000,-0.461955
2013-01-01,0.684346,-1.045700,1.981283,-1.161247
2013-01-02,-0.442428,-0.118350,0.856943,0.003684
2013-01-06,-0.568598,0.528244,-0.391605,0.638851
2013-01-04,-1.011514,1.318265,-0.640984,-0.745967
2013-01-03,-0.406470,2.594857,-1.069507,1.671639


In [10]:
%magic